# LASA recognition

## Sound-alike

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.cluster import KMeans, AffinityPropagation
import matplotlib.pyplot as plt
import nltk
from nltk.metrics.distance import edit_distance
from tqdm.notebook import tqdm
import sys
import pickle
import string
import sys, math, random, copy

In [2]:
df = pd.read_csv("./drugsatfda20211116/Products.txt", sep='\t+', engine='python')
df

,ApplNo,ProductNo,Form,Strength,ReferenceDrug,DrugName,ActiveIngredient,ReferenceStandard
0,4,4,SOLUTION/DROPS;OPHTHALMIC,1%,0,PAREDRINE,HYDROXYAMPHETAMINE HYDROBROMIDE,0.0
1,159,1,TABLET;ORAL,500MG,0,SULFAPYRIDINE,SULFAPYRIDINE,0.0
2,552,1,INJECTABLE;INJECTION,"20,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
3,552,2,INJECTABLE;INJECTION,"40,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
4,552,3,INJECTABLE;INJECTION,"5,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM,0.0
...,...,...,...,...,...,...,...,...
43206,761201,2,INJECTABLE;INJECTION,3ML(100UNITS/ML),0,SEMGLEE,INSULIN GLARGINE-YFGN,0.0
43207,761202,1,INJECTABLE;INJECTION,0.5MG(10MG/ML),0,BYOOVIZ,RANIBIZUMAB-NUNA,0.0
43208,761208,1,INJECTABLE;INJECTION,40MG,0,TIVDAK,TISOTUMAB VEDOTIN-TFTV,0.0
43209,761210,1,INJECTABLE;INJECTION,350MG/7ML(50MG/ML),0,RYBREVANT,AMIVANTAMAB-VMJW,0.0


In [3]:
drugNames = df['DrugName']
drugNames

0               PAREDRINE
1           SULFAPYRIDINE
2        LIQUAEMIN SODIUM
3        LIQUAEMIN SODIUM
4        LIQUAEMIN SODIUM
               ...       
43206             SEMGLEE
43207             BYOOVIZ
43208              TIVDAK
43209           RYBREVANT
43210            JEMPERLI
Name: DrugName, Length: 43211, dtype: object

In [4]:
drugNames = drugNames.drop_duplicates() \
                     .dropna()
random_incides = [np.random.randint(0, len(drugNames)) for _ in range(10)]
drugNames.iloc[random_incides]

1942     PLASMA-LYTE 56 AND DEXTROSE 5% IN PLASTIC CONT...
25697                                        PAPA-DEINE #3
6157                                             REMODULIN
6154                                            CARDURA XL
13686                                             ATENOLOL
1126                                        MEDROL ACETATE
43202                                           NEXVIAZYME
30007                                       ISOPTO CARPINE
29489                                               ANTHIM
1002                                   SERPASIL-ESIDRIX #2
Name: DrugName, dtype: object

In [5]:
names = np.array(drugNames)
len(names)

7572

In [6]:
# Try edit distance between different phonetic forms

# Calculate similarity matrix between letters
# https://www.diva-portal.org/smash/get/diva2:1116701/FULLTEXT01.pdf

neighbors_of = {}
# nw ne e se sw w
neighbors_of['q'] = [ 'w', 'a']
neighbors_of['w'] = [ 'e', 's', 'a', 'q']
neighbors_of['e'] = [ 'r', 'd', 's', 'w']
neighbors_of['r'] = [ 't', 'f', 'd', 'e']
neighbors_of['t'] = [ 'y', 'g', 'f', 'r']
neighbors_of['y'] = [ 'u', 'h', 'g', 't']
neighbors_of['u'] = [ 'i', 'j', 'h', 'y']
neighbors_of['i'] = [ 'o', 'k', 'j', 'u']
neighbors_of['o'] = [ 'p', 'l', 'k', 'i']
neighbors_of['p'] = [ 'l', 'o']
neighbors_of['a'] = ['q', 'w', 's', 'z']
neighbors_of['s'] = ['w', 'e', 'd', 'x', 'z', 'a']
neighbors_of['d'] = ['e', 'r', 'f', 'c', 'x', 's']
neighbors_of['f'] = ['r', 't', 'g', 'v', 'c', 'd']
neighbors_of['g'] = ['t', 'y', 'h', 'b', 'v', 'f']
neighbors_of['h'] = ['y', 'u', 'j', 'n', 'b', 'g']
neighbors_of['j'] = ['u', 'i', 'k', 'm', 'n', 'h']
neighbors_of['k'] = ['i', 'o', 'l', 'm', 'j']
neighbors_of['l'] = ['o', 'p', 'k']
neighbors_of['z'] = ['a', 's', 'x']
neighbors_of['x'] = ['s', 'd', 'c', 'z']
neighbors_of['c'] = ['d', 'f', 'v', 'x']
neighbors_of['v'] = ['f', 'g', 'b', 'c']
neighbors_of['b'] = ['g', 'h', 'n', 'v']
neighbors_of['n'] = ['h', 'j', 'm', 'b']
neighbors_of['m'] = ['j', 'k', 'n']

keys = sorted(neighbors_of.keys())
dists = {el:{} for el in keys}

# Distance between letters and their neighbours
def distance(start, end, raw):
    if start == end:
        if raw:
            return 0
        else:
            return 1
        
    visited = [start]
    queue = []
    
    for key in neighbors_of[start]:
        queue.append({'char': key, 'dist': 1})
        
    while True:
        key = queue.pop(0)
        visited.append(key['char'])
        if key['char'] == end:
            return key['dist']
        
        for neighbor in neighbors_of[key['char']]:
            if neighbor not in visited:
                queue.append({'char': neighbor, 'dist': key['dist']+1})

In [7]:
# Computes a similarity matrix for letters of the English alphabet
# based on keyboard distances between letters 
def alldists(option, verbose):
    if option == "linear":
        longest_dist = 0
        avgdist = 0
        for i in range(len(keys)):
            for j in range(len(keys)):
                dist = distance(keys[i], keys[j], False)
                dists[keys[i]][keys[j]] = 2 - (2 * dist / 9.0)
                avgdist += dists[keys[i]][keys[j]]
                if dist > longest_dist:
                    longest_dist = dist
        key_dist = longest_dist
        avgdist /= len(keys) ** 2 + 0.0
        if verbose:
            print("Average distance: " + str(avgdist))
        
        avgdisttwo = 0
        
        for i in range(len(keys)):
            for j in range(len(keys)):
                dists[keys[i]][keys[j]] /= avgdist
                avgdisttwo += dists[keys[i]][keys[j]]
                
        avgdisttwo /= len(keys) ** 2 + 0.0
        if verbose:
            print("Average distance after normalizing: " + str(avgdisttwo))
            print("Longest distance: " + str(key_dist))
            print("Longest logarithmed: " + str(math.log(key_dist)))
            print("Logarithmed & normalized: " + str(math.log(key_dist) / math.log(9)))
            print(str(dists).replace("'", '"'))
            
    elif option == "neighbors":
        longest_dist = 0
        avgdist = 0
        for i in range(len(keys)):
            for j in range(len(keys)):
                dist = distance(keys[i], keys[j], False)
                if dist == 1:
                    dists[keys[i]][keys[j]] = 2.0
                else:
                    dists[keys[i]][keys[j]] = 1.0
                avgdist += dists[keys[i]][keys[j]]
                if dist > longest_dist:
                    longest_dist = dist
        key_dist = longest_dist
        avgdist /= len(keys) ** 2 + 0.0
        if verbose:
            print("Average distance: " + str(avgdist))
            
        avgdisttwo = 0
        
        for i in range(len(keys)):
            for j in range(len(keys)):
                dists[keys[i]][keys[j]] /= avgdist 
                avgdisttwo += dists[keys[i]][keys[j]]
                
        avgdisttwo /= len(keys) ** 2 + 0.0
        if verbose:
            print("Average distance after normalizing: " + str(avgdisttwo))
            print("Longest distance: " + str(key_dist))
            print("Longest logarithmed: " + str(math.log(key_dist)))
            print("Logarithmed & normalized: " + str(math.log(key_dist) / math.log(9)))
            print(str(dists).replace("'", '"'))
            
    elif option == "raw":
        longest_dist = 0
        avgdist = 0
        for i in range(len(keys)):
            for j in range(len(keys)):
                dists[keys[i]][keys[j]] = distance(keys[i], keys[j], True)
                avgdist += dists[keys[i]][keys[j]]
                if dists[keys[i]][keys[j]] > longest_dist:
                    longest_dist = dists[keys[i]][keys[j]]
        key_dist = longest_dist
        avgdist /= len(keys) ** 2 + 0.0
        
        buckets = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        for i in range(len(keys)):
            for j in range(len(keys)):
                buckets[dists[keys[i]][keys[j]]] += 1
        if verbose:
            print("Average distance: " + str(avgdist))
            print("Longest distance: " + str(key_dist))
            print("Buckets: " + str(buckets))
            print(str(dists).replace("'", '"'))
    return copy.deepcopy(dists)

In [8]:
# Take all ascii characters
all_ascii = string.printable
print(all_ascii)

# Add the manually computed Edit Distance for letters to the full similarity matrix
# Add hardcoded similarity for the other characters (0 if same character, 12 otherwise)
similarity_dict = alldists("raw", False)
similarity_dict_all = {}

# Construct full similarity matrix by iterating through all ascii characters
for a in all_ascii:
    similarity_dict_all[a] = {}
    for b in all_ascii:
        # If characters are the same, assign 0
        # Otherwise if similarity has alredy been computed, assign that value
        # Otherwise assign 12      
        similarity_dict_all[a][b] = 0 if a == b else similarity_dict[a][b] if a in similarity_dict and b in similarity_dict[a] else 12
similarity_array = np.zeros((len(similarity_dict), len(similarity_dict)))

for character_index, (character, other_characters) in enumerate(similarity_dict.items()):
    for c_index, c in enumerate(other_characters.values()):
        similarity_array[character_index][c_index] = c

for i in similarity_dict_all:
    print(i)
    print(similarity_dict_all[i])

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

0
{'0': 0, '1': 12, '2': 12, '3': 12, '4': 12, '5': 12, '6': 12, '7': 12, '8': 12, '9': 12, 'a': 12, 'b': 12, 'c': 12, 'd': 12, 'e': 12, 'f': 12, 'g': 12, 'h': 12, 'i': 12, 'j': 12, 'k': 12, 'l': 12, 'm': 12, 'n': 12, 'o': 12, 'p': 12, 'q': 12, 'r': 12, 's': 12, 't': 12, 'u': 12, 'v': 12, 'w': 12, 'x': 12, 'y': 12, 'z': 12, 'A': 12, 'B': 12, 'C': 12, 'D': 12, 'E': 12, 'F': 12, 'G': 12, 'H': 12, 'I': 12, 'J': 12, 'K': 12, 'L': 12, 'M': 12, 'N': 12, 'O': 12, 'P': 12, 'Q': 12, 'R': 12, 'S': 12, 'T': 12, 'U': 12, 'V': 12, 'W': 12, 'X': 12, 'Y': 12, 'Z': 12, '!': 12, '"': 12, '#': 12, '$': 12, '%': 12, '&': 12, "'": 12, '(': 12, ')': 12, '*': 12, '+': 12, ',': 12, '-': 12, '.': 12, '/': 12, ':': 12, ';': 12, '<': 12, '=': 12, '>': 12, '?': 12, '@': 12, '[': 12, '\\': 12, ']': 12, '^': 12, '_': 12, '`': 12, '{': 12, '|': 12, '}': 12, '~': 12, ' ': 12, '\t': 12, '\n': 12, '\r': 12, '\x0b': 12, 

In [9]:
ins_cost = 1
del_cost = 1

def min_cost_path(cost, operations):
    
    # operation at the last cell
    path = [operations[cost.shape[0]-1][cost.shape[1]-1]]
    
    # cost at the last cell
    min_cost = cost[cost.shape[0]-1][cost.shape[1]-1]
    
    row = cost.shape[0]-1
    col = cost.shape[1]-1
    
    while row > 0 and col > 0:
            
        if cost[row-1][col-1] <= cost[row-1][col] and cost[row-1][col-1] <= cost[row][col-1]:
            path.append(operations[row-1][col-1])
            row -= 1
            col -= 1
        elif cost[row-1][col] <= cost[row-1][col-1] and cost[row-1][col] <= cost[row][col-1]:
            path.append(operations[row-1][col])
            row -= 1
        else:
            path.append(operations[row][col-1])
            col -= 1
                    
    return "".join(path[::-1][1:])

def edit_distance_dp(seq1, seq2):
    # there is no difference between upper and lower case for this application    
    seq1 = seq1.lower()
    seq2 = seq2.lower()
    
    # create an empty 2D matrix to store cost
    cost = np.zeros((len(seq1)+1, len(seq2)+1))
    
    # fill the first row
    cost[0] = [i for i in range(len(seq2)+1)]
    
    # fill the first column
    cost[:, 0] = [i for i in range(len(seq1)+1)]
    
    # to store the operations made
    operations = np.asarray([['-' for j in range(len(seq2)+1)] \
                                 for i in range(len(seq1)+1)])
    
    # fill the first row by insertion 
    operations[0] = ['I' for i in range(len(seq2)+1)]
    
    # fill the first column by insertion operation (D)
    operations[:, 0] = ['D' for i in range(len(seq1)+1)]
    
    operations[0, 0] = '-'
    
    # now, iterate over earch row and column
    for row in range(1, len(seq1)+1):
        
        for col in range(1, len(seq2)+1):
            
            # if both the characters are same then the cost will be same as 
            # the cost of the previous sub-sequence
            if seq1[row-1] == seq2[col-1]:
                cost[row][col] = cost[row-1][col-1]
            else:
                
                insertion_cost = cost[row][col-1] + ins_cost
                deletion_cost = cost[row-1][col] + del_cost
                substitution_cost = cost[row-1][col-1] + similarity_dict_all[seq1[row-1]][seq2[col-1]]
#                 print(f"sim for {seq1[row-1]} and {seq2[col-1]}: {similarity_dict_all[seq1[row-1]][seq2[col-1]]}")
                
                # calculate the minimum cost
                cost[row][col] = min(insertion_cost, deletion_cost, substitution_cost)
                
                # get the operation
                if cost[row][col] == substitution_cost:
                    operations[row][col] = 'S'
                    
                elif cost[row][col] == ins_cost:
                    operations[row][col] = 'I'
                else:
                    operations[row][col] = 'D'
    return cost[len(seq1), len(seq2)], min_cost_path(cost, operations)

# seq1 = "numpy~!@~!~!@"
# seq2 = "numexpr~~!!*&"
# score, operations = edit_distance_dp(seq1, seq2)
# print(f"Edit Distance between `{seq1}` & `{seq2}` is: {score}")
# print("\nOperations performed are:\n")
# for operation in operations:
#     if operation == '-':
#         print('No Change.')
#     elif operation == 'I':
#         print('Insertion')
#     elif operation == 'D':
#         print('Deletion')
#     else:
#         print('Substitution')

In [10]:
# Levenshtein distance
# n = len(names)
n = 300
lev_dist = np.zeros((n, n))
lev_sim = np.zeros((n, n))

for i in tqdm(range(n)):
    for j in range(i+1, n):
        ni = names[i]
        nj = names[j]
        dist, operations = edit_distance_dp(ni, nj)
        lev_dist[i, j] = dist
        lev_dist[j, i] = dist

  0%|          | 0/300 [00:00<?, ?it/s]

In [11]:
def is_row_similar(row, threshold=10):
    sorted_row = sorted(row)[:len(row)//4]
#     lowest = sorted(row)[1]
#     return lowest < threshold
    return np.average(sorted_row) < threshold

In [12]:
# file_path = 'lev_dist3000.pickle'
# file_path2 = 'lev_dist3000_3.pickle'
# pickle.dump(lev_dist, open(file_path, "wb"))
# lev_dist = pickle.load(open(file_path2, "rb"))
# print(np.shape(lev_sim))

filter_lev_dist = []
columns_to_remove = []
for i, row in enumerate(lev_dist):
    if is_row_similar(row):
        filter_lev_dist.append(row)  
    else:
        columns_to_remove.append(i)

# filter_lev_dist = []
# columns_to_remove = []
# for i, row in enumerate(lev_dist):
#     filter_lev_dist.append(row)  

for i, row in enumerate(filter_lev_dist):
    filter_lev_dist[i] = [entry for c, entry in enumerate(row) if c not in columns_to_remove]
      
filter_lev_dist = np.array(filter_lev_dist)
print(np.shape(filter_lev_dist))
# np.savetxt('filter_lev_sim.txt',filter_lev_sim,fmt='%s')

(198, 198)


In [13]:
# Distance to similarity
# Try out other ways to translate distance to similarity

# lev_sim = filter_lev_sim
lev_sim = 1 / (1 + filter_lev_dist)
lev_sim
# np.savetxt('lev_sim.txt',lev_sim,fmt='%.2f')

array([[1.        , 0.09090909, 0.1       , ..., 0.09090909, 0.1       ,
        0.08333333],
       [0.09090909, 1.        , 0.125     , ..., 0.11111111, 0.125     ,
        0.11111111],
       [0.1       , 0.125     , 1.        , ..., 0.11111111, 0.09090909,
        0.1       ],
       ...,
       [0.09090909, 0.11111111, 0.11111111, ..., 1.        , 0.1       ,
        0.09090909],
       [0.1       , 0.125     , 0.09090909, ..., 0.1       , 1.        ,
        0.08333333],
       [0.08333333, 0.11111111, 0.1       , ..., 0.09090909, 0.08333333,
        1.        ]])

In [14]:
# Cluster on computed similarities
aff_prop = AffinityPropagation(affinity="precomputed", damping=0.96, verbose=True)
aff_prop.fit(lev_sim);
print(f'Found {len(aff_prop.cluster_centers_indices_)} clusters.')

Converged after 110 iterations.
Found 34 clusters.


C:\Users\danie\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(


In [15]:
for cluster_id in range(len(aff_prop.cluster_centers_indices_)):
    exemplar = names[aff_prop.cluster_centers_indices_[cluster_id]]
    members = names[np.nonzero(aff_prop.labels_ == cluster_id)]

    print(f'{cluster_id + 1}. \033[1m{exemplar}\033[0m ({len(members)} members): {", ".join(members)}')


1. SULFAPYRIDINE (9 members): SULFAPYRIDINE, ORETON METHYL, SYNKAYVITE, DIETHYLSTILBESTROL, PANTOPAQUE, VERTAVIS, NORISODRINE, TAPAZOLE, CORTEF
2. HISTAMINE PHOSPHATE (8 members): HISTAMINE PHOSPHATE, HYCODAN, MANNITOL 25%, D.H.E. 45, BENEMID, EVANS BLUE, ACTH, THIOSULFIL
3. INULIN AND SODIUM CHLORIDE (5 members): INULIN AND SODIUM CHLORIDE, GANTRISIN, BERUBIGEN, PRANTAL, HEDULIN
4. BENSULFOID (3 members): BENSULFOID, SULFONAMIDES DUPLEX, PRO-BANTHINE
5. STILBESTROL (9 members): VERARD, MENADIONE, METANDREN, STILBESTROL, AMINOHIPPURATE SODIUM, DIENESTROL, FOLIC ACID, PERCODAN, DIBENZYLINE
6. STILBETIN (6 members): STILBETIN, HEAVY SOLUTION NUPERCAINE, PARADIONE, AZULFIDINE, LEUCOVORIN CALCIUM, RAUWILOID
7. PERCORTEN (16 members): THEELIN, DEMEROL, PERCORTEN, LYNORAL, DICUMAROL, BAL, MESANTOIN, DOLOPHINE HYDROCHLORIDE, METUBINE IODIDE, PHENURONE, NEOTHYLLINE, SUS-PHRINE SULFITE FREE, PHENERGAN W/ CODEINE, BENTYL PRESERVATIVE FREE, LEVO-DROMORAN, SUCOSTRIN
8. PANHEPRIN (11 members): PANH